This notebook will clean, process, and analyze raw data collected via an automated web scraper, providing insights into pricing strategies across brands and price segments

In [37]:
# Importing libraries 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from ipywidgets import interact, widgets

In [38]:
# Importing dataset
data = pd.read_csv(r"C:\Users\chels\ALex beginner\Python\Flipkart_Mobile_Prices_Multiple_Page.csv")
pd.set_option('display.max.rows', 1000)
data.head(1000)

# Renaming the Price column as Sale_Price to differentiate between prices during and before the sale
data.rename(columns = {'Price' : 'Sale_Price'}, inplace = True)

In [40]:
# Checking datatypes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12672 entries, 0 to 12671
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        12672 non-null  object
 1   Time        12672 non-null  object
 2   Product     12648 non-null  object
 3   Sale_Price  12648 non-null  object
dtypes: object(4)
memory usage: 396.1+ KB


The data reveals null values in the Product and Sale_Price columns, which we will investigate further. 
Since the Sale_Price column is stored as a string, we will update and store it as an Integer
Additionally, the Date and Time columns are stored as strings; we will merge them into a single column and convert the datatype to Datetime

In [41]:
# Deleting null values in Product and Sale_Price
data[data['Product'].isnull()]
data.dropna(inplace = True)

In [42]:
# CONVERTING DATA TYPES & NORMALISING DATA

#  PRICE
# Removing '₹' AND ',' and then storing the the values as Integers
data['Sale_Price'] = data['Sale_Price'].str[1:].str.replace(',','').astype('int')


# DATE
data.Date.unique()

# We notice 2 different date formats
data[(data['Date']== '26-01-2024') | (data['Date'] == '2024-01-26')]
# The first 95 entries have the issue

# Unifying the time format
data.Date = data.Date.str.replace('26-01-2024', '2024-01-26')
data.Date.unique()


# TIME
data.Time.unique() 
# We remove the Time post minutes
data['Time'] = data['Time'].str[0:6]
data.Time.unique() 

# Merging Date and Time into a single column and storing it as DateTime
data['Datetime'] = data.Date + data.Time
data['Datetime'] = pd.to_datetime(data['Datetime'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12648 entries, 0 to 12671
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        12648 non-null  object        
 1   Time        12648 non-null  object        
 2   Product     12648 non-null  object        
 3   Sale_Price  12648 non-null  int32         
 4   Datetime    12648 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(1), object(3)
memory usage: 543.5+ KB


In [43]:
data.describe()

,Sale_Price,Datetime
count,12648.000000,12648
mean,17221.556135,2024-02-01 04:42:05.123339520
min,4999.000000,2024-01-26 00:06:00
25%,8399.000000,2024-01-26 20:47:00
50%,12999.000000,2024-01-29 13:12:00
75%,19255.500000,2024-02-07 10:04:00
max,76999.000000,2024-02-12 11:46:00
std,14353.253754,NaN


In [25]:
# Making a list of Brands in the dataset
data.Product.str
.split().str[0].unique()


array(['Apple', 'Motorola', 'vivo', 'POCO', 'realme', 'Infinix',
       'SAMSUNG', 'REDMI', 'OnePlus', 'OPPO', 'Nothing', 'MOTOROLA'],
      dtype=object)

On exploring the dataset, we notice, that the same products of the same brands have different prices on the same dates and at the same time.

On manually checking, we notice that they are prices of a specific smartphone but with different specifications i.e. the price difference is because of the difference in RAM specifications or SSD sepcifications.

Since we cannot ascertain which price is for what model, we remove this data from our analysis

In [44]:
# Same Products that have more than 1 price

#  Grouping by Product and Datetime, following by counting number of unique prices
price_count = data.groupby(['Product', 'Datetime'])['Sale_Price'].nunique()
# Products with more than 1 unique price at the same date and time
price_count = price_count[price_count > 1]

# List of products to be deleted 
del_list = price_count.reset_index().Product.unique()

In [48]:
# Finding the index, of the products to be deleted, in the main df
del_list = data[data['Product'].isin(del_list)].index

# Deleting products using the above obtained index data
data.drop(del_list, inplace = True)

Some products maintain the same price throughout the sale period. Since we want to analyse products with fluctuating prices during the sale period, we remove these products with constant prices.

In [49]:
# Products that have more than 2 prices
price_count = data.groupby('Product')['Sale_Price'].nunique()
price_count = price_count[price_count == 1]

# List of products to be deleted 
del_list = price_count.reset_index().Product.unique()

# Finding the index, of the products to be deleted, in the main df
del_list = data[data['Product'].isin(del_list)].index
# Deleting products using the above obtained index data
data.drop(del_list, inplace = True)

In [50]:
# Adding Price column - this indicates prices before Sale
data['Price']= None

In [51]:
# Adding Pre-sale prices for all products 
price_data = {'Apple iPhone 13 (Blue, 128 GB)': 52999, 
'Apple iPhone 13 (Midnight, 128 GB)': 52990,
'Apple iPhone 13 (Pink, 128 GB)': 52999,
'Apple iPhone 13 (Starlight, 128 GB)': 52999,
'Apple iPhone 14 (Blue, 128 GB)' : 59999,
'Apple iPhone 14 (Midnight, 128 GB)':59999,
'Apple iPhone 14 (Purple, 128 GB)': 59999,
'Apple iPhone 14 (Starlight, 128 GB)':59999,
'Apple iPhone 15 (Black, 128 GB)':77900,
'Apple iPhone 15 (Blue, 128 GB)':77900,
'Apple iPhone 15 (Green, 128 GB)':75999,
'Apple iPhone 15 (Pink, 128 GB)':75999,
'Apple iPhone 15 Plus (Black, 128 GB)':82999,
'Apple iPhone 15 Plus (Blue, 128 GB)':82999,
'Apple iPhone 15 Plus (Pink, 128 GB)':82999,
'Motorola Edge 40 (Nebula Green, 256 GB)': 26999, 
'Motorola Edge 40 Neo (Black Beauty, 128 GB)': 22999,
'Motorola g84 5G (Viva Magenta, 256 GB)' : 18999,
'POCO C51 (Power Black, 64 GB)': 5999, 
'POCO C51 (Royal Blue, 64 GB)': 5999,
'POCO C51 - Locked with Airtel Prepaid (Royal Blue, 64 GB)': 5499,
'POCO C55 (Cool Blue, 64 GB)': 6499,
'POCO C55 (Forest Green, 128 GB)' : 7499,
'POCO C55 (Power Black, 128 GB)':7499,
'POCO C55 (Power Black, 64 GB)': 6499,
'POCO M4 5G (Power Black, 128 GB)':10999,
'POCO M4 5G (Yellow, 128 GB)':10999,
'POCO M6 5G (Galactic Black, 128 GB)':11499,
'POCO M6 Pro 5G (Forest Green, 256 GB)':13499,
'POCO M6 Pro 5G (Power Black, 256 GB)':13499,
'POCO X5 5G (Supernova Green, 128 GB)':13999,
'realme 11 5G (Glory Black, 128 GB)': 16999, 
'realme 11 5G (Glory Gold, 128 GB)': 16999,
'realme C51 (Mint Green, 64 GB)': 8999,
'realme C53 (Champion Black, 64 GB)': 10999,
'realme C53 (Champion Gold, 64 GB)': 9999,
'Infinix HOT 30i (Diamond White, 128 GB)': 8299, 
'Infinix HOT 30i (Diamond White, 64 GB)': 7499,
'Infinix HOT 30i (Glacier Blue, 64 GB)': 7499,
'Infinix HOT 30i (Marigold, 64 GB)': 7499,
'Infinix HOT 30i (Mirror Black, 128 GB)': 8299,
'Infinix HOT 30i (Mirror Black, 64 GB)':7499,
'Infinix SMART 8 (Galaxy White, 64 GB)': 6299,
'Infinix SMART 8 (Rainbow Blue, 64 GB)':7499,
'Infinix SMART 8 (Shiny Gold, 64 GB)':7499,
'Infinix SMART 8 (Timber Black, 64 GB)':7499,
'Infinix SMART 8 HD (Crystal Green, 64 GB)':6299,
'Infinix SMART 8 HD (Galaxy White, 64 GB)':6299,
'Infinix SMART 8 HD (Shiny Gold, 64 GB)':6299,
'SAMSUNG Galaxy F04 (Jade Purple, 64 GB)': 11499, 
'SAMSUNG Galaxy F04 (Opal Green, 64 GB)': 7999,
'SAMSUNG Galaxy F13 (Nightsky Green, 64 GB)': 14999,
'SAMSUNG Galaxy F13 (Sunrise Copper, 64 GB)': 10999,
'SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB)': 14999,
'SAMSUNG Galaxy M04 (Dark Blue, 64 GB)':7550,
'SAMSUNG Galaxy M04 (Light Green, 64 GB)': 7490,
'SAMSUNG Galaxy M14 5G (Berry Blue, 128 GB)':12290,
'SAMSUNG Galaxy M14 5G (Icy Silver, 128 GB)':11900,
'REDMI 13c 5G (Startrail Green, 256 GB)' : 14499, 
'REDMI 13c 5G (Startrail Silver, 128 GB)': 15999,
'OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)': 17673, 
'OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)': 18570,
'OnePlus Nord CE 3 Lite 5G (Pastel lime, 256 GB)': 21383}

for product, price in price_data.items():
    data.loc[data['Product'] == product, 'Price'] = price

In [52]:
# Changing it's data type from Integer
data['Price'] = data['Price'].astype('int')

In [53]:
# Adding a Discount column to capture price differences
data['Discount'] = data['Price'] - data['Sale_Price']

In [54]:
# Making separate dfs for each brand
apple = data[data.Product.str.contains('Apple')]
motorola = data[data.Product.str.contains('Motorola', case = False)]
vivo = data[data.Product.str.contains('vivo')]
poco = data[data.Product.str.contains('POCO')]
realme = data[data.Product.str.contains('realme')] 
infinix = data[data.Product.str.contains('Infinix')] 
samsung = data[data.Product.str.contains('SAMSUNG')]
redmi = data[data.Product.str.contains('REDMI')]
oneplus = data[data.Product.str.contains('OnePlus')]
oppo = data[data.Product.str.contains('OPPO')]
nothing = data[data.Product.str.contains('Nothing')]

In [55]:
brands = [apple,motorola,poco,realme,infinix,samsung,redmi,oneplus]
for brand in brands:
    print(brand.info())
# No null values in any of the brand dfs 

# Sorting each brand df by Product, Date and Time
brand_names = ['apple', 'motorola', 'poco', 'realme', 'infinix', 'samsung', 'redmi', 'oneplus']

for brand, name in zip(brands,brand_names):
    exec(f"{name} = brand.sort_values(['Product', 'Date', 'Time'])")

<class 'pandas.core.frame.DataFrame'>
Index: 818 entries, 0 to 12655
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        818 non-null    object        
 1   Time        818 non-null    object        
 2   Product     818 non-null    object        
 3   Sale_Price  818 non-null    int32         
 4   Datetime    818 non-null    datetime64[ns]
 5   Price       818 non-null    int32         
 6   Discount    818 non-null    int32         
dtypes: datetime64[ns](1), int32(3), object(3)
memory usage: 41.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 130 entries, 280 to 12635
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        130 non-null    object        
 1   Time        130 non-null    object        
 2   Product     130 non-null    object        
 3   Sale_Price  130 non-null    int32         
 

Next, we will analyze the data to uncover pricing strategies by examining how brands adjust their pricing during and after the sale period.

In [56]:
brand_names = ['apple', 'motorola', 'poco', 'realme', 'infinix', 'samsung', 'redmi', 'oneplus']

# Finding out Maxiumun and Minimum discount for each product
for brand, name in zip(brands,brand_names):
    exec(f"{name}_discount = brand.groupby('Product').agg({{'Discount': [min, max]}})")
    # Dropping level 0 index
    exec(f"{name}_discount.columns = {name}_discount.columns.droplevel(0)")
    # Renaming columns
    exec(f"{name}_discount.rename(columns=({{'min': 'min_discount','max': 'max_discount'}}),inplace = True)")
    # Adding additional columns in brand_discount dfs
    exec(f"{name}_discount[['min_date','max_date']] = None")
    # Creating a dictionary of Product names and Discount
    exec(f"{name}_discount_dict = {name}_discount.to_dict()")

    
    # Adding dates with minimum discount using the brand_discount dictionaries created
    exec(f"""for x,y in {name}_discount_dict['min_discount'].items():
        dates = {name}[({name}['Discount'] == y) & ({name}['Product'] == x)].Date.unique()
        {name}_discount.loc[x,'min_date'] = ', '.join(map(str, dates))""")
    
    #Adding dates with maximum discount using the brand_discount dictionaries created
    exec(f"""for x,y in {name}_discount_dict['max_discount'].items():
        dates = {name}[({name}['Discount'] == y) & ({name}['Product'] == x)].Date.unique()
        {name}_discount.loc[x,'max_date'] = ', '.join(map(str, dates))""")

Analyzing the pricing strategies of different brands using graphical and tabular visualizations.

In [58]:
# APPLE

# Graphical Visualisation 

# Defining the interactive function
def plot_product(product):
    # Filtering data according to selected product
    product_data = apple[apple['Product'] == product]
    
    # Creating the graph
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=product_data['Datetime'],
        y=product_data['Sale_Price'],
        mode='lines+markers',
        name= product
    ))
    
    # Formating the X-axis
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=product_data['Datetime'],  
            ticktext=[f"{time} \n{date}" for date, time in zip(product_data['Date'], product_data['Time'])],  
        )
    )

    fig.show()
    
# Creating a dropdown
product_dropdown = widgets.Dropdown(
    options=apple['Product'].sort_values().unique(),
    description='Product:',
    disabled=False,
)


# Linking dropdown to plot function
interact(plot_product, product=product_dropdown)

interactive(children=(Dropdown(description='Product:', options=('Apple iPhone 14 (Blue, 128 GB)', 'Apple iPhon…

<function __main__.plot_product(product)>

In [59]:
# Tabular visualisation
apple_discount

,min_discount,max_discount,min_date,max_date
Product,,,,
"Apple iPhone 14 (Blue, 128 GB)",1000,4000,"2024-02-05, 2024-02-06, 2024-02-07","2024-01-30, 2024-01-31"
"Apple iPhone 14 (Midnight, 128 GB)",1000,4000,"2024-02-05, 2024-02-06, 2024-02-07","2024-01-30, 2024-01-31"
"Apple iPhone 14 (Purple, 128 GB)",1000,4000,"2024-02-05, 2024-02-07","2024-01-30, 2024-01-31"
"Apple iPhone 14 (Starlight, 128 GB)",1000,4000,"2024-02-05, 2024-02-06, 2024-02-07","2024-01-30, 2024-01-31"
"Apple iPhone 15 (Black, 128 GB)",4901,11901,"2024-02-06, 2024-02-07","2024-01-28, 2024-01-29, 2024-01-30, 2024-02-09"
"Apple iPhone 15 (Blue, 128 GB)",4901,11901,"2024-02-05, 2024-02-06, 2024-02-07","2024-01-28, 2024-01-29, 2024-01-30, 2024-02-09"
"Apple iPhone 15 (Green, 128 GB)",8000,10000,2024-01-26,2024-02-09
"Apple iPhone 15 (Pink, 128 GB)",6000,10000,2024-01-31,"2024-01-28, 2024-01-29, 2024-01-30, 2024-02-09"


In [60]:
print('apple_discount')
print(apple_discount['max_date'].str.split(', ').explode().value_counts())
print(apple_discount['min_date'].str.split(', ').explode().value_counts())

apple_discount
max_date
2024-01-30    7
2024-01-31    4
2024-02-09    4
2024-01-28    3
2024-01-29    3
Name: count, dtype: int64
min_date
2024-02-07    6
2024-02-05    5
2024-02-06    5
2024-01-26    1
2024-01-31    1
Name: count, dtype: int64


In [24]:
# MOTOROLA

# Graphical Visualisation 

# Defining the interactive function
def plot_product(product):
    # Filtering data according to selected product
    product_data = motorola[motorola['Product'] == product]
    
    # Creating the graph
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=product_data['Datetime'],
        y=product_data['Sale_Price'],
        mode='lines+markers',
        name= product
    ))
    # Formatingthe X-axis
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=product_data['Datetime'], 
            ticktext=[f"{time} \n{date}" for date, time in zip(product_data['Date'], product_data['Time'])],  
            title='Date and Time'
        )
    )
    fig.show()
    
# Creating a dropdown
product_dropdown = widgets.Dropdown(
    options=motorola['Product'].sort_values().unique(),
    description='Product:',
    disabled=False,
)


# Linking dropdown to plot function
interact(plot_product, product=product_dropdown)

interactive(children=(Dropdown(description='Product:', options=('Motorola Edge 40 (Nebula Green, 256 GB)', 'Mo…

<function __main__.plot_product(product)>

In [25]:
# Tabular visualisation
motorola_discount

,min_discount,max_discount,min_date,max_date
Product,,,,
"Motorola Edge 40 (Nebula Green, 256 GB)",0,19700,"2024-01-30, 2024-01-31, 2024-02-05, 2024-02-06...",2024-02-07
"Motorola Edge 40 Neo (Black Beauty, 128 GB)",-4000,0,2024-02-07,"2024-01-26, 2024-01-28, 2024-01-31, 2024-02-05..."
"Motorola g84 5G (Viva Magenta, 256 GB)",0,1000,2024-01-28,"2024-02-05, 2024-02-06, 2024-02-07, 2024-02-09..."


In [26]:
print('motorola_discount')
print(motorola_discount['max_date'].str.split(', ').explode().value_counts())
print(motorola_discount['min_date'].str.split(', ').explode().value_counts())

motorola_discount
max_date
2024-02-07    3
2024-02-05    2
2024-02-06    2
2024-02-09    2
2024-01-26    1
2024-01-28    1
2024-01-31    1
2024-02-11    1
2024-02-12    1
Name: count, dtype: int64
min_date
2024-02-07    2
2024-01-30    1
2024-01-31    1
2024-02-05    1
2024-02-06    1
2024-02-09    1
2024-02-11    1
2024-01-28    1
Name: count, dtype: int64


In [27]:
# Poco

# Graphical Visualisation 

# Defining the interactive function
def plot_product(product):
    # Filtering data according to selected product
    product_data = poco[poco['Product'] == product]
    
    # Creating the graph
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=product_data['Datetime'],
        y=product_data['Sale_Price'],
        mode='lines+markers',
        name= product
    ))
    # Formating the X-axis
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=product_data['Datetime'],  
            ticktext=[f"{time} \n{date}" for date, time in zip(product_data['Date'], product_data['Time'])],  
            title='Date and Time'
        )
    )
    fig.show()
    
# Creating a dropdown
product_dropdown = widgets.Dropdown(
    options=poco['Product'].sort_values().unique(),
    description='Product:',
    disabled=False,
)


# Linking dropdown to plot function
interact(plot_product, product=product_dropdown)

interactive(children=(Dropdown(description='Product:', options=('POCO C51 (Power Black, 64 GB)', 'POCO C51 (Ro…

<function __main__.plot_product(product)>

In [28]:
# Tabular visualisation
poco_discount

,min_discount,max_discount,min_date,max_date
Product,,,,
"POCO C51 (Power Black, 64 GB)",300,500,"2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29...","2024-02-05, 2024-02-06, 2024-02-07, 2024-02-09..."
"POCO C51 (Royal Blue, 64 GB)",200,500,"2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29","2024-02-05, 2024-02-06, 2024-02-07, 2024-02-09..."
"POCO C51 - Locked with Airtel Prepaid (Royal Blue, 64 GB)",100,500,"2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29...","2024-02-05, 2024-02-06, 2024-02-07"
"POCO C55 (Cool Blue, 64 GB)",0,200,"2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29...","2024-02-05, 2024-02-06, 2024-02-07, 2024-02-09..."
"POCO C55 (Forest Green, 128 GB)",0,200,"2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29...","2024-02-05, 2024-02-06, 2024-02-07, 2024-02-09..."
"POCO C55 (Power Black, 128 GB)",0,200,"2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29...","2024-02-05, 2024-02-06, 2024-02-07, 2024-02-09..."
"POCO C55 (Power Black, 64 GB)",0,200,"2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29...","2024-02-05, 2024-02-06, 2024-02-07, 2024-02-09..."
"POCO M4 5G (Power Black, 128 GB)",-4000,1000,2024-02-09,"2024-02-05, 2024-02-07"
"POCO M4 5G (Yellow, 128 GB)",-4000,1000,"2024-02-09, 2024-02-11","2024-02-05, 2024-02-07"


In [29]:
print('poco_discount')
print(poco_discount['max_date'].str.split(', ').explode().value_counts())
print(poco_discount['min_date'].str.split(', ').explode().value_counts())

poco_discount
max_date
2024-02-07    12
2024-02-05    11
2024-02-06     9
2024-02-09     8
2024-02-11     8
2024-02-12     6
2024-01-26     1
2024-01-27     1
2024-01-28     1
2024-01-29     1
Name: count, dtype: int64
min_date
2024-01-26    10
2024-01-27     8
2024-01-28     8
2024-01-29     7
2024-01-30     6
2024-01-31     5
2024-02-09     3
2024-02-11     1
2024-02-07     1
Name: count, dtype: int64


In [30]:
# Realme

# Graphical Visualisation 

# Defining the interactive function
def plot_product(product):
    # Filtering data according to selected product
    product_data = realme[realme['Product'] == product]
    
    # Creating the graph
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=product_data['Datetime'],
        y=product_data['Sale_Price'],
        mode='lines+markers',
        name= product
    ))
    # Formating the X-axis
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=product_data['Datetime'],  
            ticktext=[f"{time} \n{date}" for date, time in zip(product_data['Date'], product_data['Time'])], 
            title='Date and Time'
        )
    )
    fig.show()
    
# Creating a dropdown
product_dropdown = widgets.Dropdown(
    options=realme['Product'].sort_values().unique(),
    description='Product:',
    disabled=False,
)


# Linking dropdown to plot function
interact(plot_product, product=product_dropdown)

interactive(children=(Dropdown(description='Product:', options=('realme 11 5G (Glory Black, 128 GB)', 'realme …

<function __main__.plot_product(product)>

In [31]:
# Tabular visualisation
realme_discount

,min_discount,max_discount,min_date,max_date
Product,,,,
"realme 11 5G (Glory Black, 128 GB)",-8000,1000,2024-02-07,"2024-02-09, 2024-02-11, 2024-02-12"
"realme 11 5G (Glory Gold, 128 GB)",0,1000,"2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29...","2024-02-09, 2024-02-11, 2024-02-12"
"realme C51 (Mint Green, 64 GB)",0,1000,"2024-01-26, 2024-01-27, 2024-01-28","2024-02-05, 2024-02-12"
"realme C53 (Champion Black, 64 GB)",0,2000,"2024-02-05, 2024-02-06, 2024-02-07","2024-02-11, 2024-02-12"
"realme C53 (Champion Gold, 64 GB)",-1000,1000,"2024-02-05, 2024-02-06, 2024-02-07","2024-02-11, 2024-02-12"


In [32]:
print('realme_discount')
print(realme_discount['max_date'].str.split(', ').explode().value_counts())
print(realme_discount['min_date'].str.split(', ').explode().value_counts())

realme_discount
max_date
2024-02-12    5
2024-02-11    4
2024-02-09    2
2024-02-05    1
Name: count, dtype: int64
min_date
2024-02-07    4
2024-02-05    3
2024-02-06    3
2024-01-26    2
2024-01-27    2
2024-01-28    2
2024-01-29    1
2024-01-31    1
Name: count, dtype: int64


In [33]:
# Infinix

# Graphical Visualisation 

# Defining the interactive function
def plot_product(product):
    # Filtering data according to selected product
    product_data = infinix[infinix['Product'] == product]
    
    # Creating the graph
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=product_data['Datetime'],
        y=product_data['Sale_Price'],
        mode='lines+markers',
        name= product
    ))
    # Formating the X-axis
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=product_data['Datetime'], 
            ticktext=[f"{time} \n{date}" for date, time in zip(product_data['Date'], product_data['Time'])],  
            title='Date and Time'
        )
    )
    fig.show()
    
# Creating a dropdown
product_dropdown = widgets.Dropdown(
    options=infinix['Product'].sort_values().unique(),
    description='Product:',
    disabled=False,
)


# Linking dropdown to plot function
interact(plot_product, product=product_dropdown)

interactive(children=(Dropdown(description='Product:', options=('Infinix HOT 30i (Diamond White, 128 GB)', 'In…

<function __main__.plot_product(product)>

In [34]:
# Tabular visualisation
infinix_discount

,min_discount,max_discount,min_date,max_date
Product,,,,
"Infinix HOT 30i (Diamond White, 128 GB)",-1200,0,2024-02-09,"2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29..."
"Infinix HOT 30i (Diamond White, 64 GB)",-900,-200,"2024-02-05, 2024-02-07, 2024-02-09, 2024-02-11","2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29..."
"Infinix HOT 30i (Glacier Blue, 64 GB)",-900,-200,"2024-02-05, 2024-02-07, 2024-02-09, 2024-02-11","2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29..."
"Infinix HOT 30i (Marigold, 64 GB)",-900,-200,"2024-02-05, 2024-02-07, 2024-02-09, 2024-02-11","2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29..."
"Infinix HOT 30i (Mirror Black, 128 GB)",-1200,0,2024-02-09,"2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29..."
"Infinix HOT 30i (Mirror Black, 64 GB)",-900,-200,"2024-02-05, 2024-02-07, 2024-02-09, 2024-02-11","2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29..."
"Infinix SMART 8 (Galaxy White, 64 GB)",-1000,-800,"2024-01-29, 2024-01-30, 2024-01-31",2024-02-05
"Infinix SMART 8 (Rainbow Blue, 64 GB)",200,400,"2024-01-29, 2024-01-30, 2024-01-31",2024-02-05
"Infinix SMART 8 (Shiny Gold, 64 GB)",200,400,"2024-01-29, 2024-01-30, 2024-01-31",2024-02-05


In [35]:
print('infinix_discount')
print(infinix_discount['max_date'].str.split(', ').explode().value_counts())
print(infinix_discount['min_date'].str.split(', ').explode().value_counts())

infinix_discount
max_date
2024-01-26    9
2024-01-27    9
2024-01-28    9
2024-01-29    9
2024-02-05    9
2024-01-31    8
2024-01-30    6
2024-02-07    5
2024-02-06    3
2024-02-09    2
2024-02-11    2
Name: count, dtype: int64
min_date
2024-02-09    9
2024-02-11    7
2024-02-05    4
2024-02-07    4
2024-01-29    4
2024-01-30    4
2024-01-31    4
2024-02-12    3
Name: count, dtype: int64


In [36]:
# Samsung

# Graphical Visualisation 

# Defining the interactive function
def plot_product(product):
    # Filtering data according to selected product
    product_data = samsung[samsung['Product'] == product]
    
    # Creating the graph
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=product_data['Datetime'],
        y=product_data['Sale_Price'],
        mode='lines+markers',
        name= product
    ))
    # Formating the X-axis
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=product_data['Datetime'],  
            ticktext=[f"{time} \n{date}" for date, time in zip(product_data['Date'], product_data['Time'])],  
            title='Date and Time'
        )
    )
    # Show the plot
    fig.show()
    
# Creating a dropdown
product_dropdown = widgets.Dropdown(
    options=samsung['Product'].sort_values().unique(),
    description='Product:',
    disabled=False,
)


# Linking dropdown to plot function
interact(plot_product, product=product_dropdown)

interactive(children=(Dropdown(description='Product:', options=('SAMSUNG Galaxy F04 (Jade Purple, 64 GB)', 'SA…

<function __main__.plot_product(product)>

In [37]:
# Tabular visualisation
samsung_discount

,min_discount,max_discount,min_date,max_date
Product,,,,
"SAMSUNG Galaxy F04 (Jade Purple, 64 GB)",3500,5500,"2024-01-29, 2024-01-30, 2024-01-31","2024-01-26, 2024-01-27, 2024-01-28, 2024-02-12"
"SAMSUNG Galaxy F04 (Opal Green, 64 GB)",0,2000,"2024-01-29, 2024-01-30, 2024-01-31","2024-01-26, 2024-01-27, 2024-01-28, 2024-02-12"
"SAMSUNG Galaxy F13 (Nightsky Green, 64 GB)",4000,7500,2024-01-31,"2024-01-26, 2024-01-28, 2024-01-29, 2024-01-30..."
"SAMSUNG Galaxy F13 (Sunrise Copper, 64 GB)",0,3500,2024-01-31,"2024-01-26, 2024-01-27, 2024-01-28, 2024-01-29..."
"SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB)",4000,7500,2024-01-31,"2024-01-26, 2024-01-28, 2024-01-29, 2024-01-30..."
"SAMSUNG Galaxy M04 (Dark Blue, 64 GB)",670,740,2024-02-05,2024-02-09
"SAMSUNG Galaxy M04 (Light Green, 64 GB)",541,702,2024-02-07,2024-02-05
"SAMSUNG Galaxy M14 5G (Berry Blue, 128 GB)",845,899,2024-02-09,2024-02-09
"SAMSUNG Galaxy M14 5G (Icy Silver, 128 GB)",-797,-418,2024-01-26,2024-02-05


In [38]:
print('samsung_discount')
print(samsung_discount['max_date'].str.split(', ').explode().value_counts())
print(samsung_discount['min_date'].str.split(', ').explode().value_counts())

samsung_discount
max_date
2024-01-26    5
2024-01-28    5
2024-02-12    4
2024-01-27    3
2024-01-29    3
2024-01-30    3
2024-02-09    2
2024-02-05    2
Name: count, dtype: int64
min_date
2024-01-31    5
2024-01-29    2
2024-01-30    2
2024-02-05    1
2024-02-07    1
2024-02-09    1
2024-01-26    1
Name: count, dtype: int64


In [39]:
# Redmi

# Graphical Visualisation 

# Defining the interactive function
def plot_product(product):
     # Filtering data according to selected product
    product_data = redmi[redmi['Product'] == product]
    
    # Creating the graph
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=product_data['Datetime'],
        y=product_data['Sale_Price'],
        mode='lines+markers',
        name= product
    ))
    # Formating the X-axis
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=product_data['Datetime'], 
            ticktext=[f"{time} \n{date}" for date, time in zip(product_data['Date'], product_data['Time'])],  
            title='Date and Time'
        )
    )
    fig.show()
    
# Creating a dropdown
product_dropdown = widgets.Dropdown(
    options=redmi['Product'].sort_values().unique(),
    description='Product:',
    disabled=False,
)


# Linking dropdown to plot function
interact(plot_product, product=product_dropdown)

interactive(children=(Dropdown(description='Product:', options=('REDMI 13c 5G (Startrail Green, 256 GB)', 'RED…

<function __main__.plot_product(product)>

In [40]:
# Tabular visualisation
redmi_discount

,min_discount,max_discount,min_date,max_date
Product,,,,
"REDMI 13c 5G (Startrail Green, 256 GB)",284,305,2024-02-09,2024-02-07
"REDMI 13c 5G (Startrail Silver, 128 GB)",3440,3451,2024-02-09,2024-02-09


In [41]:
print('redmi_discount')
print(redmi_discount['max_date'].str.split(', ').explode().value_counts())
print(redmi_discount['min_date'].str.split(', ').explode().value_counts())

redmi_discount
max_date
2024-02-07    1
2024-02-09    1
Name: count, dtype: int64
min_date
2024-02-09    2
Name: count, dtype: int64


In [42]:
# Oneplus

# Graphical Visualisation 

# Defining the interactive function
def plot_product(product):
     # Filtering data according to selected product
    product_data = oneplus[oneplus['Product'] == product]
    
    # Creating the graph
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=product_data['Datetime'],
        y=product_data['Sale_Price'],
        mode='lines+markers',
        name= product
    ))
    # Formating the X-axis
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=product_data['Datetime'],  
            ticktext=[f"{time} \n{date}" for date, time in zip(product_data['Date'], product_data['Time'])],  
            title='Date and Time'
        )
    )
    fig.show()
    
# Creating a dropdown
product_dropdown = widgets.Dropdown(
    options=oneplus['Product'].sort_values().unique(),
    description='Product:',
    disabled=False,
)


# Linking dropdown to plot function
interact(plot_product, product=product_dropdown)

interactive(children=(Dropdown(description='Product:', options=('OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB…

<function __main__.plot_product(product)>

In [43]:
# Tabular visualisation
oneplus_discount

,min_discount,max_discount,min_date,max_date
Product,,,,
"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)",-146,-96,2024-02-09,2024-02-11
"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)",112,655,2024-02-07,2024-01-31
"OnePlus Nord CE 3 Lite 5G (Pastel lime, 256 GB)",84,805,2024-02-07,2024-02-05


In [44]:
print('oneplus_discount')
print(oneplus_discount['max_date'].str.split(', ').explode().value_counts())
print(oneplus_discount['min_date'].str.split(', ').explode().value_counts())

oneplus_discount
max_date
2024-02-11    1
2024-01-31    1
2024-02-05    1
Name: count, dtype: int64
min_date
2024-02-07    2
2024-02-09    1
Name: count, dtype: int64


A price analysis of the Republic Day Sale reveals that, out of several brands, most Samsung and Infinix smartphones experience the highest price reductions on 26th January, contrary to the expectation of peak discounts occurring on the festive date.

Most brands tend to offer higher discounts in February, post the sale period. While Motorola and Poco maintain maximum discounts on all the February dates, Apple, Realme, and Samsung concentrate their highest reductions towards the latter days. This may be due to low demand and high inventory.

Infinix deviates from this trend, with its highest price reductions occurring in January, followed closely by Samsung. 
Additionally, brands like Apple, OnePlus, and Infinix exhibit significant discounts on the 30th and 31st of January.

Next, we will analyze price data of smartphone across brands of various price segments to uncover trends (if any) during and after the sale period.

In [134]:
# We divide the products into the following price ranges:
r1 = data[data['Price'] < 10000]
r2 = data[(data['Price'] > 10000) & (data['Price'] < 20000)]
r3 = data[(data['Price'] > 20000) & (data['Price'] < 30000)]
r4 = data[(data['Price'] > 30000)]



ranges = ['r1','r2','r3', 'r4']
# Finding out Maxiumun and Minimum discount for each product in each price range
for name in ranges:
    exec(f"{name}_discount = {name}.groupby('Product').agg({{'Discount': [min, max]}})")

    # Dropping level 0 index
    exec(f"{name}_discount.columns = {name}_discount.columns.droplevel(0)")
    # Renaming columns
    exec(f"{name}_discount.rename(columns=({{'min': 'min_discount','max': 'max_discount'}}),inplace = True)")
    # Adding additional columns in range_discount dfs
    exec(f"{name}_discount[['min_date','max_date']] = None")
    # Creating a dictionary of Product names and Discount
    exec(f"{name}_discount_dict = {name}_discount.to_dict()")
    
    
    # Adding dates with minimum discount using the range_discount dictionaries created
    exec(f"""for x,y in {name}_discount_dict['min_discount'].items():
        dates = {name}[({name}['Discount'] == y) & ({name}['Product'] == x)].Date.unique()
        {name}_discount.loc[x,'min_date'] = ', '.join(map(str, dates))""")
    
    #    #Adding dates with maximum discount using the range_discount dictionaries created
    exec(f"""for x,y in {name}_discount_dict['max_discount'].items():
        dates = {name}[({name}['Discount'] == y) & ({name}['Product'] == x)].Date.unique()
        {name}_discount.loc[x,'max_date'] = ', '.join(map(str, dates))""")

In [135]:
# Printing the data

print('Models less than 10,000')
print(r1_discount['max_date'].str.split(', ').explode().value_counts())
print(r1_discount['min_date'].str.split(', ').explode().value_counts())

print('Models between 10,000 - 20,000')
print(r2_discount['max_date'].str.split(', ').explode().value_counts())
print(r2_discount['min_date'].str.split(', ').explode().value_counts())

print('Models between 20,000 - 30,000')
print(r3_discount['max_date'].str.split(', ').explode().value_counts())
print(r3_discount['min_date'].str.split(', ').explode().value_counts())


print('Models greater than 30,000')
print(r4_discount['max_date'].str.split(', ').explode().value_counts())
print(r4_discount['min_date'].str.split(', ').explode().value_counts())


# print('Models between 20,000 - 25,000')
# print(r4_discount['max_date'].str.split(', ').explode().value_counts())
# print(r4_discount['min_date'].str.split(', ').explode().value_counts())

# print('Models between 25,000 - 30,000')
# print(r5_discount['max_date'].str.split(', ').explode().value_counts())
# print(r5_discount['min_date'].str.split(', ').explode().value_counts())


Models less than 10,000
max_date
2024-02-05    18
2024-02-07    12
2024-01-26    10
2024-01-27    10
2024-01-28    10
2024-02-06    10
2024-01-29     9
2024-02-09     9
2024-02-11     9
2024-01-31     8
2024-01-30     6
2024-02-12     6
Name: count, dtype: int64
min_date
2024-01-29    12
2024-01-30    11
2024-01-31    10
2024-02-09     9
2024-01-26     8
2024-01-27     8
2024-01-28     8
2024-02-11     7
2024-02-05     6
2024-02-07     6
2024-02-12     3
2024-02-06     1
Name: count, dtype: int64
Models between 10,000 - 20,000
max_date
2024-02-12    10
2024-02-07     7
2024-02-09     7
2024-02-05     6
2024-02-11     6
2024-01-26     5
2024-01-28     5
2024-01-29     4
2024-02-06     3
2024-01-27     3
2024-01-30     3
2024-01-31     1
Name: count, dtype: int64
min_date
2024-02-09    7
2024-02-07    5
2024-01-26    5
2024-01-31    5
2024-01-28    3
2024-01-27    2
2024-01-29    2
2024-02-05    2
2024-02-06    2
2024-02-11    1
2024-01-30    1
Name: count, dtype: int64
Models between 20

Price range analysis reveals distinct discount trends across different smartphone segments:

    1. Budget models (< ₹10,000) experience the highest discounts on 5th and 7th February, followed by 26th–28th January, while minimal discounts occur on 29th–31st January. This suggests that sellers aim to clear inventory as demand declines post-sale.
    2. Mid-range models (₹10,000 – ₹20,000) follow a similar pattern, with discounts peaking in February, followed by January. Interestingly, dates like 7th and 9th February show both maximum and minimum discounts, indicating that discounts may vary by time of day.
    3. Upper mid-range models (₹20,000 – ₹30,000) exhibit the same trend but with fewer discount days.
    4. Premium models (> ₹30,000) receive minimal discounts in February, but have significant price reductions concentrated at the end of January.
    
As smartphone prices increase, discounts become less frequent and shorter in duration. Low-end phones see maximum discounts in February, whereas premium models, particularly Apple smartphones, receive discounts mainly at the end of January.